# Lista 5

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.signal import chirp, square, sawtooth, unit_impulse, hilbert

### Zadanie 1
Przygotuj w Pythonie kod, który wyznaczy widmo amplitudowe sygnału sinusoidalnego.  

Jeżeli sygnał poddawany transformacji Fouriera jest rzeczywisty, wtedy zawsze otrzymamy dwie kopie widma, druga kopia będzie lustrzanym odbiciem pierwszej (dokładnie: będzie jej zespolonym sprzężeniem). Często potrzebujemy tylko jednej kopii widma. Funkcja np.fft.rfft oblicza widmo rzeczywistego sygnału. Funkcja np.fft.fftfreq oblicza częstotliwość dla każdego punktu widmowego, argumentami funkcji są: liczba wartości widma (długość transformaty) oraz okres próbkowania. Ponadto skalujemy widmo dzieląc je przez połowę długości transformaty, aby wartości widmowe reprezentowały energię poszczególnych składowych (przez połowę długości, ponieważ energia sygnału rozkłada się równomiernie na dwie części widma, a my obserwujemy tylko jedną z nich).

In [2]:
def amplituda_sinus(f=5, fs=1000):
    
    t = np.linspace(0, 1, fs, endpoint=False)
    sin_wave = np.sin(2 * np.pi * f * t)
    
    widmo = np.fft.fft(sin_wave)
    widmo_amp_pelne = np.abs(widmo)

    widmo_amp_rzecz = np.abs(np.fft.rfft(sin_wave)) / (len(sin_wave)/2)
    f_widmo = np.fft.rfftfreq(len(sin_wave), 1/fs)

    plt.figure(figsize=(8, 6))
    plt.scatter(t, sin_wave, color='red')
    plt.plot(t, sin_wave, linewidth=1)
    plt.xlabel("Czas [s]")
    plt.ylabel("Amplituda")
    plt.title(f"Sygnał sinusoidalny: f={f}Hz, fs={fs}Hz")
    plt.grid()
    

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(widmo_amp_pelne)
    plt.xlabel('indeks widma')
    plt.ylabel('amplituda widma')
    plt.title(f'Pełne widmo sygnału sinusoidalnego f={f}Hz')
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.plot(f_widmo, widmo_amp_rzecz)
    plt.xlabel('częstotliwość [Hz]')
    plt.ylabel('amplituda widma')
    plt.title(f'Widmo "rzeczywiste" sygnału sinusoidalnego f={f}Hz')
    plt.grid()
    plt.tight_layout()
    
    


f_slider = widgets.IntSlider(min=1, max=20, step=1, value=5, description="f [Hz]")
fs_slider = widgets.IntSlider(min=10, max=1000, step=10, value=200, description="fs [Hz]")

display(widgets.interactive(amplituda_sinus, f=f_slider, fs=fs_slider))

interactive(children=(IntSlider(value=5, description='f [Hz]', max=20, min=1), IntSlider(value=200, descriptio…

### Zadanie 2
Przygotuj w Pythonie kod, który wyznaczy widmo amplitudowe sygnału sinusoidalnego w skali decybelowej. 

In [3]:
def amplituda_sinus_decybel(f=5, fs=1000):
    
    t = np.linspace(0, 1, fs, endpoint=False)
    sin_wave = np.sin(2 * np.pi * f * t)
    

    widmo_amp_rzecz = np.abs(np.fft.rfft(sin_wave)) / len(sin_wave) * 2
    f_widmo = np.fft.rfftfreq(len(sin_wave), 1/fs)
    widmo_amp_rzecz_db = 20 * np.log10(np.maximum(widmo_amp_rzecz, 1e-12))


    plt.figure(figsize=(8, 6))
    plt.plot(f_widmo, widmo_amp_rzecz_db)
    plt.xlabel('częstotliwość [Hz]')
    plt.ylabel('amplituda widma [dB]')
    plt.title(f'Widmo sygnału sinusoidalnego f={f}Hz - skala decybelowa')
    plt.grid()


display(widgets.interactive(amplituda_sinus_decybel, f=f_slider, fs=fs_slider))

interactive(children=(IntSlider(value=5, description='f [Hz]', max=20, min=1), IntSlider(value=200, descriptio…

### Zadanie 3
Przygotuj w Pythonie kod, który wyznaczy widmo amplitudowe sygnałów z zadania 1 na liście 1. 

In [4]:
def amplituda_sinus(f=5, fs=1000):
    
    t = np.linspace(0, 1, fs, endpoint=False)
    
    signals = {
    'Sinus' : np.sin(2 * np.pi * f * t),
    'Prostokątny' : square(2 * np.pi * f * t),
    'Piłokształtny' : sawtooth(2 * np.pi * f * t),
    'Świergotliwy' : chirp(t, f0=5, f1=2*f, t1=1, method='linear'),
    'Superpozycja sinus cosinus' : np.sin(2 * np.pi * f * t) + 0.5 * np.cos(2 * np.pi * 2*f * t),
    'Impuls jednostkowy' : unit_impulse(fs, idx=fs//2)
    }

    
    
    for (sig_label, signal) in signals.items():
        widmo = np.fft.fft(signal)
        widmo_amp_pelne = np.abs(widmo)

        widmo_amp_rzecz = np.abs(np.fft.rfft(signal)) / len(signal) * 2
        f_widmo = np.fft.rfftfreq(len(signal), 1/fs)
        
        # Tworzenie wykresów
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        axes.flatten()

        axes[0].plot(t, signal, linewidth=1)
        axes[0].set_xlabel("Czas [s]")
        axes[0].set_ylabel("Amplituda")
        axes[0].set_title(f"Sygnał {sig_label}: f0={5}Hz, f1={2*f}Hz fs={fs}Hz" if sig_label=='Świergotliwy' 
                          else f"Sygnał {sig_label}: f={f}Hz, fs={fs}Hz")
        axes[0].grid()
        
        axes[1].plot(widmo_amp_pelne)
        axes[1].set_xlabel('indeks widma')
        axes[1].set_ylabel('amplituda widma')
        axes[1].set_title(f'Pełne widmo sygnału')
        axes[1].grid()

        
        axes[2].plot(f_widmo, widmo_amp_rzecz)
        axes[2].set_xlabel('częstotliwość [Hz]')
        axes[2].set_ylabel('amplituda widma')
        axes[2].set_title(f'Widmo "rzeczywiste" sygnału')
        axes[2].grid()
        fig.tight_layout()
    
    


f_slider = widgets.IntSlider(min=1, max=20, step=1, value=10, description="f [Hz]")
fs_slider = widgets.IntSlider(min=100, max=1000, step=10, value=300, description="fs [Hz]")

display(widgets.interactive(amplituda_sinus, f=f_slider, fs=fs_slider))

interactive(children=(IntSlider(value=10, description='f [Hz]', max=20, min=1), IntSlider(value=300, descripti…

### Zadanie 4
Przygotuj w Pythonie kod, który wyznaczy widmo fazowe sygnału sinusoidalnego.

In [5]:
def widmo_fazowe(f=5, fs=1000):

    t = np.linspace(0, 1, fs, endpoint=False)
    sin_wave = np.sin(2 * np.pi * f * t)    

    widmo_faz = np.angle(np.fft.rfft(sin_wave))
    f_widmo = np.fft.rfftfreq(len(sin_wave), 1/fs)

    plt.plot(f_widmo, widmo_faz)
    plt.xlabel('częstotliwość [Hz]')
    plt.ylabel('faza [rad]')
    plt.title('Widmo fazowe sygnału sinusoidalnego')

f_slider = widgets.IntSlider(min=1, max=20, step=1, value=10, description="f [Hz]")
fs_slider = widgets.IntSlider(min=10, max=1000, step=10, value=300, description="fs [Hz]")

display(widgets.interactive(widmo_fazowe, f=f_slider, fs=fs_slider))
    

interactive(children=(IntSlider(value=10, description='f [Hz]', max=20, min=1), IntSlider(value=300, descripti…

### Zadanie 5
Przygotuj w Pythonie kod, który wyznaczy obwiednie dla sygnału świergotliwego (chirp signal)

Wyznaczam obwiednię za pomocą transformacji Hilberta. Obwiednia jest obliczana jako moduł wartości zespolonej uzyskanej z sygnału.

In [ ]:
def obwiednia_chirp(f=5, fs=1000):
    
    t = np.linspace(0, 1, fs, endpoint=False)
    chirp_signal = chirp(t, f0=5, f1=3*f, t1=1, method='linear')

    transf_hilberta = hilbert(chirp_signal)
    obwiednia = np.abs(transf_hilberta)

    plt.figure(figsize=(10, 5))
    plt.plot(t, chirp_signal, label=f'Sygnał świergotliwy: : f0={5}Hz, f1={3*f}Hz', alpha=0.7)
    plt.plot(t, obwiednia, 'r', label='Obwiednia', linewidth=2)
    plt.xlabel('Czas [s]')
    plt.ylabel('Amplituda')
    plt.title('Obwiednia sygnału świergotliwego')
    plt.legend()
    plt.grid()

f_slider = widgets.IntSlider(min=1, max=20, step=1, value=10, description="f [Hz]")
fs_slider = widgets.IntSlider(min=10, max=1000, step=10, value=300, description="fs [Hz]")

display(widgets.interactive(obwiednia_chirp, f=f_slider, fs=fs_slider))

interactive(children=(IntSlider(value=10, description='f [Hz]', max=20, min=1), IntSlider(value=300, descripti…